In [1]:
import numpy as np
import pandas as pd

In [20]:
pd.options.display.max_rows = 200

In [19]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

In [9]:
emb_dim = 50

In [7]:
model = Sequential()

In [10]:
model.add(Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=input_length))
model.add(LSTM(1000, activation='relu', return_sequences=True))
model.add(LSTM(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 50)            50000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 1000)          4204000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 1000)              8004000   
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              1001000   
Total params: 14,260,000
Trainable params: 14,260,000
Non-trainable params: 0
_________________________________________________________________
